In [4]:
import json

dataset = [json.loads(line) for line in open("sentencedataset.json")]
dataset[:5]

[{'sentence_id': 0,
  'paragraph_id': 0,
  'story_id': 0,
  'sentence': 'Katniss Everdeen and Peta had two kids and their names were Rye and Willow.',
  'paragraph_index': 0,
  'sentence_index': 0,
  'title': '015501',
  'source': 'ChiSCor',
  'URL': 'https://osf.io/5h7za/?view_only=705c553e922046058ef9df52b4ac8ed7'},
 {'sentence_id': 1,
  'paragraph_id': 0,
  'story_id': 0,
  'sentence': 'and one day they sat on the couch with each other and watched team titans.',
  'paragraph_index': 0,
  'sentence_index': 1,
  'title': '015501',
  'source': 'ChiSCor',
  'URL': 'https://osf.io/5h7za/?view_only=705c553e922046058ef9df52b4ac8ed7'},
 {'sentence_id': 2,
  'paragraph_id': 0,
  'story_id': 0,
  'sentence': 'but then the tv came on and they saw commander Paylor.',
  'paragraph_index': 0,
  'sentence_index': 2,
  'title': '015501',
  'source': 'ChiSCor',
  'URL': 'https://osf.io/5h7za/?view_only=705c553e922046058ef9df52b4ac8ed7'},
 {'sentence_id': 3,
  'paragraph_id': 0,
  'story_id': 0,
  's

In [20]:
import random
import numpy as np

from collections import defaultdict

# Merge all the sentence from the same paragraphs
sent_reorder_data = defaultdict(list)

for instance in dataset:
  sent_reorder_data[instance["paragraph_id"]].append(instance)

# Filter out single sentence paragraphs
sent_reorder_data = {paragraph_id: instances
                     for paragraph_id, instances in sent_reorder_data.items()
                     if len(instances) > 2}

# Create a shuffled example from each paragraph
shuffled_data = []
for instances in sent_reorder_data.values():
  random.shuffle(instances)
  shuffled_data.append(instances)

with open("shuffled_sent_data.jsonl", "w") as f_out:
  for instances in shuffled_data:

    # Get the sorted instances based on sentence_index
    sentence_indices = [instance["sentence_index"] + 1 for instance in instances]
    order = np.argsort(sentence_indices)
    order = [i+1 for i in order]
    example = {"label": str(order),
               "list_of_sentences": [instance["sentence"] for instance in instances]}
    f_out.write(json.dumps(example) + "\n")

In [1]:
!pip install transformers

In [ ]:
# from transformers import AutoTokenizer
# from transformers import AutoModelForCausalLM

MODEL_NAME = "google/flan-t5-large"

# model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)
# tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

from transformers import T5Tokenizer, T5ForConditionalGeneration

tokenizer = T5Tokenizer.from_pretrained(MODEL_NAME)
model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


config.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.13G [00:00<?, ?B/s]

In [5]:
# Load the dataset
dataset = [json.loads(line) for line in open("shuffled_sent_data.jsonl")]
print(f"Data size: {len(dataset)}, First example: {dataset[0]}")

Data size: 721, First example: {'label': '[13, 12, 1, 9, 6, 5, 8, 4, 7, 2, 10, 14, 11, 3]', 'list_of_sentences': ['but then the tv came on and they saw commander Paylor.', 'so they decided to climb trees and found a luxurious cave to stay in.', 'and in the end they won.', "in the blood bath that first day they killed a lot of people themselves but also tried to run out so they wouldn't die themselves.", 'so then when the day came they went into the arena and when the gong sounded they went and grabbed all supplies they needed.', "and everyone was surprised when she said that the victor's kids would have to go back in the arena again with their parents.", 'it was a forest arena the same one that Katniss and Peta were used to.', 'bows and arrows spears knives tridents and apparently a frying pan.', 'and she announced that there was going to be a seventy sixth hunger games.', 'they got many sponsors because Katniss and Peta were very famous for the capital people.', "so they decided to st

In [ ]:
def generate_text(tokenizer, model, prompt):
  # model_inputs = tokenizer([prompt], return_tensors="pt").to("cuda")
  # generated_ids = model.generate(**model_inputs)
  # return tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
  input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")
  generated_ids = model.generate(input_ids)
  return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

In [7]:
prompt = """You're given a shuffled story and you need to output the order of the original story. The output should be in the format 4, 2, 11, etc. based on the given numbering.\n"""

def reorder_story(example):
   curr_prompt = prompt + "\n".join([f"{i+1}. {sent}" for i, sent in enumerate(example["list_of_sentences"])])
   print(curr_prompt)
   output = generate_text(tokenizer, model, curr_prompt)
   print(output)

reorder_story(dataset[0])

You're given a shuffled story and you need to output the order of the original story. The output should be in the format 4, 2, 11, etc. based on the given numbering.
1. but then the tv came on and they saw commander Paylor.
2. so they decided to climb trees and found a luxurious cave to stay in.
3. and in the end they won.
4. in the blood bath that first day they killed a lot of people themselves but also tried to run out so they wouldn't die themselves.
5. so then when the day came they went into the arena and when the gong sounded they went and grabbed all supplies they needed.
6. and everyone was surprised when she said that the victor's kids would have to go back in the arena again with their parents.
7. it was a forest arena the same one that Katniss and Peta were used to.
8. bows and arrows spears knives tridents and apparently a frying pan.
9. and she announced that there was going to be a seventy sixth hunger games.
10. they got many sponsors because Katniss and Peta were very 

Setting `pad_token_id` to `eos_token_id`:0 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1178: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


ValueError: Input length of input_ids is 345, but `max_length` is set to 20. This can lead to unexpected behavior. You should consider increasing `max_length` or, better yet, setting `max_new_tokens`.